# GRIPFEB21

# Sparks Foundation Internship
# Name : Abhisarika Tidke
# Domain : Computer Vision and Internet Of Things
# Task 1 : Object Detection(Basic Level)

In [6]:
#importing libraries
from cv2 import cv2
import numpy as np

In [14]:
net = cv2.dnn.readNet('yolov3.weights', 'yolov3.cfg')#this network contains yolov3 weights and configuration files
classes=[]
with open('coco.Names', 'r') as f:
    classes = f.read().splitlines()
cap = cv2.VideoCapture('tm.mp4')#video file
#img = cv2.imread('dw.png')

In [15]:
while True:
    _,img = cap.read()
    height, width, _ = img.shape
    #normalizing the image by dividing by factor of 255
    blob = cv2.dnn.blobFromImage(img, 1/255,(416, 416), (0,0,0), swapRB=True,crop=False)
    net.setInput(blob)#used to set the input from the blob into the network
    output_layers_names = net.getUnconnectedOutLayersNames()
    layerOutputs = net.forward(output_layers_names)#Used to run the forward pass and obtainoutput at op layer
    
    #now creation of bounded box and confidence value
    boxes=[]#to extract the bounding boxes
    confidences = []#to extract the confidence
    class_ids = []#for classes id which represent predicted classes
    #4 bounding box offset, 1 confidence and 80 class predictions(i.e. total 85 parameters)
    for output in layerOutputs:#to extract info from output layers
        for detection in output:#to extract info from output
            scores = detection[5:]#5th element is confidence. After that is 80 classes so it stores 80 class predictions
            class_id = np.argmax(scores)#extracts highest score location
            confidence = scores[class_id]#highest score is assigned to confidence
            if confidence > 0.5:
                center_x = int(detection[0]*width)
                center_y = int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                x = int(center_x - w/2)#position of upper left corner
                y = int(center_y - h/2)

                boxes.append([x, y, w, h])

                confidences.append((float(confidence)))

                class_ids.append(class_id)


    #print(len(boxes))#prints number of object classes in the image
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)
    #print(indexes.flatten())#prints number of boxes
    font = cv2.FONT_HERSHEY_PLAIN
    colors = np.random.uniform(0,255, size=(len(boxes), 3))
    for i in indexes.flatten():
        x, y, w, h = boxes[i]
        label = str(classes[class_ids[i]])
        confidence = str(round(confidences[i],2))
        color = colors[i]
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        cv2.putText(img, label + " " + confidence, (x, y+20), font, 2, (255, 255, 255), 2)




    cv2.imshow('Image',img)
    key = cv2.waitKey(1)
    if key == 27:
        break
cap.release()
cv2.destroyAllWindows()